## 	Метод интеграции графа с фильтрацией контекста 

Графовая структура данных позволяет иметь доступ к расположению каждого из узлов графа в карте знаний программных продуктов. Эти знания будут использоваться для того, чтобы сообщать LLM о положении каждого из чанков в контексте — это позволит LLM определять какой из чанков релевантный ответу, а какой нет.  Для извлечения расположения узла в графе знаний используются cypher запрос - Листинг 6. 

Листинг 6. Cypher запрос для извлечения пути к вершине p в документе. 
```
f"MATCH p=(:Root_node)-[*]->(r) WHERE r.id='{node.id}' RETURN p"

```
В реализации вопросно-ответных систем распространен метод ветвления [23]. Ветвление в этом методе реализовано путем введения специальной фильтрации найденных узлов графа алгоритмом поиска, таким образом, чтобы LLM, отвечая на вопрос, получала в контекст информацию только об одном программном продукте. Это позволит не допускать попадание ложного контекста в подсказку к LLM.  Для этого будет использоваться фильтрация путем удаления выбросов в контексте. Выбросами будем называть чанки, которые не относятся к области знаний задаваемого вопроса. Обычно их значительно меньше, чем релевантных чанков.


Опишем основные изменения в модулях построения вопросно-ответной системы:
- **Модуль индексации и подготовки базы знаний:** аналогичный наивному методу, за исключением того, что в метаданных узлов типа чанк или параграф добавлено поле с путем к данному узлу в документе, например чанк с информацией о системных требованиях имеет путь: дозор /2 назначение и основные возможности пп/2 2 технические требования/операционная система.
Модуль поиска: является таким же как в предыдущем методе, за исключением того, что теперь возвращает большее количество чанков, а затем фильтрует их.

- **Модуль анализа результатов поиска разделен на процессы:**
1.	Исключение выбросов из алгоритмов поиска:
Вычисляем чанки какого программного продукта встречаются в данном результате поиска чаще всего и оставляем только их. Другие чанки, относящиеся к другим программным продуктам, удаляем из контекста.

2.	Поиск дополнительных данных используя структуру графа:
1.	Находятся все списки, принадлежащие найденному узлу, и помещаются в контекст.
2.	Находятся соседние чанки для каждого узла и помещаются в контекст, аналогично предыдущему методу.
Модуль анализа результатов поиска: 
Из результатов поиска в виде чанков формируется контекст для LLM. Каждый чанк, добавленный в контекст, подписывается информацией о своем расположении в документе.
Таким образом применяя идеи, описанные в данном методе, удалось снизить процент ошибок до 10%. 

## Устанавливаем api ключи и прокси 

In [ ]:
# run examples/graph_creation/docs2graph_neo4j_advanced.py
# db sandbox will be saved in neo4j
db_name = 'sandbox'

In [ ]:
import os
# setting for langsmith
os.environ["LANGCHAIN_PROJECT"] = "graph-rag"
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_PROJECT"] = "graph-rag"
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_API_KEY"] = "<>"

# setting for openai
os.environ["OPENAI_API_KEY"] = "<>"
os.environ["OPENAI_BASE_URL"] = 'https://api.proxyapi.ru/openai/v1'

# setting for neo4j
os.environ["NEO4J_URI"] ="bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "login"
os.environ["NEO4J_PASSWORD"] = "pass"


In [3]:
from langchain_core.prompts import ChatPromptTemplate

## Загружаем документы
## Индексируем наши чанки 

In [5]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="to_indexing",
    text_node_properties=["tl_dr", "text", "id"],
    embedding_node_property="embedding",
    database=db_name,
    index_name='main_index'
)

In [6]:
search_engine = vector_index.similarity_search_with_score("скан-архив", k=9)

In [7]:
search_engine[0][0].page_content

'\ntl_dr: Группа, где можно объединить разделы; архив - сжатая копия файлов; архивирование с паролем для защиты данных; информационная база SQL - база данных; раздел - набор данных для архивирования; расписание архивирования - задание времени и даты для создания архивов; комментарий к архиву - описание содержимого; ручное архивирование по требованию пользователя; удаленное управление для управления архиватором на другом компьютере.\ntext: гэндальф хранитель v/3 концепция программы/3 1 основные понятия<root->В приведенном ниже списке описаны основные понятия, использованные в программе и в данном Руководстве. В этот список включены как специфические термины «Хранителя V», так и общепринятые понятия, знание которых необходимо для настройки и корректного использования программы. Термины расположены в алфавитном порядке.Автоматическое архивирование. Для любого раздела можно установить режим автоматического архивирования, и указать расписание архивирования. Когда наступает время очередного 

## Загружаем LLM

In [8]:
from langchain_openai import ChatOpenAI
llm_gpt = ChatOpenAI(base_url='https://api.proxyapi.ru/openai/v1')

## Строим prompt-инструкцию для LLM

In [131]:
prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении заданий по поиску ответов на вопросы. Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос. Если вы не знаете ответа, просто скажите, что вы не знаете. 
           Вопрос: {question} 
           Контекст: {context} 
           Ответ:""")

prompt2 = ChatPromptTemplate.from_template(
           """
           Вы являетесь помощником в выполнении заданий по поиску ответов на вопросы по ПП.
           Перепишите вопрос в понятной форме так, чтобы было очевидно, где искать на него ответ.
           Вопрос: Как активировать лицензию Дозор.
           Перефразированный вопрос: Инструкция по активации ПП Дозор, как активировать программму Дозор?
           
           Вопрос: {question} 
           Перефразированный вопрос:""")

In [132]:
from langchain_core.output_parsers import StrOutputParser
question_adaptation_chain = (
    prompt2 | llm_gpt | StrOutputParser()
)

In [133]:
question_adaptation_chain.invoke('Мне хватит 8гб оперативки для дозор')

'Достаточно ли 8 гб оперативной памяти для работы программы Дозор?'

## Собираем RAG-CHAIN


In [260]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(database='method-2')

In [261]:
retriever = vector_index.as_retriever(search_kwargs={'k': 10})

In [262]:
from langchain_core.runnables import RunnablePassthrough
import re

def most_frequent_string(string_list):
    # Создаем словарь для хранения частоты каждой строки
    frequency_dict = {}
    
    # Считаем частоты строк в массиве
    for s in string_list:
        if s in frequency_dict:
            frequency_dict[s] += 1
        else:
            frequency_dict[s] = 1
            
    # Находим строку с максимальной частотой
    most_frequent = max(frequency_dict, key=frequency_dict.get)
    
    return most_frequent

def parse_to_dict(input_str):
    # Создаем пустой словарь для хранения результатов
    result_dict = {}
    is_header=False
    
    # Используем регулярные выражения для поиска всех пар ключ-значение
    if not ('\ntl_dr: \ntext:' in input_str):
        pattern = r'\n(tl_dr|text|id):\s*([^\n]+)'
    else:
        is_header=True
        pattern = r'\n(text|id):\s*([^\n]+)'
    entries = re.findall(pattern, input_str)

    # Заполняем словарь найденными парами ключ-значение
    for key, value in entries:
        result_dict[key.strip()] = value.strip()
    
    return result_dict, is_header

def get_relevant_docs(docs):
    names_pp = []
    for doc in docs:
        meta, is_header = parse_to_dict(doc.page_content)
    
        root = meta['text'].split('<root->')[0]
        
        pp_product_name = root.split('/')[0]
        names_pp.append(pp_product_name)
        
    most_frequent_pp = most_frequent_string(names_pp)      
    relevant_pp_docs =  [doc for doc in docs if most_frequent_pp in doc.page_content]

    return relevant_pp_docs[:4]

def format_docs(docs):
    exist_root = []
    context = []
    for doc in docs:
        #print(doc)
        meta, is_header = parse_to_dict(doc.page_content)
   
        #print(meta)
        root = meta['text'].split('<root->')[0]
        text =  meta['text'].split('<root->')[-1]

        if not (root in exist_root):   
            
            #print('Нашел, ', doc.page_content)
            text = "информация из раздела: Програмный продукт " + root + "\n" + text
            #print(is_header)
            
            if not is_header:
                context.append(text)
                exist_root.append(root)
            
             # Содержит текущий узел  лист, если содержит - показываем
            list_contains = graph.query(f"""
                                        MATCH p=(startNode)-[]->(list_node:List_node) WHERE startNode.id='{meta['id']}' RETURN list_node
                                        """)
            for list_node in list_contains:
                l_t = list_node['list_node']['text'].split('<root->')[-1]
                l_r = list_node['list_node']['text'].split('<root->')[0]
                text = "информация из списка: Програмный продукт " + l_r + "\n" + l_t
                context.append(text)
                
            
            # тепрь найдем соседей на 3 близкие чанки к текущему
            neighbord = graph.query(f"""
                                    MATCH path = (current)-[r*1..8]-(neighbor:Chunk_node)
                                    WHERE current.id = '{meta['id']}'
                                    AND NONE(rel IN relationships(path) WHERE type(rel) IN ['MENTION', 'MENTIONS', 'PP_CONTAINS'])
                                    WITH neighbor, length(path) AS distance
                                    ORDER BY distance
                                    RETURN neighbor                                    
                                    """)
                           
            cur_len_root = len(root.split('/'))
            for ne in neighbord[:4]:
                c_t = ne['neighbor']['text'].split('<root->')[-1] # берем текст 
                c_r = ne['neighbor']['text'].split('<root->')[0] # берем путь
                c_r_len = len(c_r.split('/'))
                
                if (not c_r in exist_root):  
                    exist_root.append(c_r)
                    text = "информация из раздела: Програмный продукт " + c_r + "\n" + c_t
                    context.append(text)
                    
    print(10*'==')
    data_split = "\n\n".join(context[:5]).split(" ")
    #print(" ".join(data_split))
    print(len(context))
    return " ".join(data_split)

In [263]:
rag_chain = (
    {"context": retriever | get_relevant_docs | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_gpt
    | StrOutputParser()
)

In [264]:
rag_chain.invoke('Мне хватит 2гб оперативки для Дозор?')

10


'Для установки и работы программного продукта "Дозор. Мониторинг систем безопасности рабочей станции" рекомендуется иметь оперативную память объемом 8 Гб. Таким образом, для работы программы будет достаточно 2 Гб оперативной памяти.'

## Считаем метрики 

### собираем ответы на тест

In [ ]:
import pandas as pd
test_set = pd.read_csv('/Applications/programming/kg_llm/assets/test_set_40.csv')

test_set['answer'] = test_set['question'].apply(lambda x: rag_chain.invoke(x.strip()))
test_set['contexts'] = test_set['contexts'].apply(lambda x:[x])
test_set.to_csv("method_2.csv", index=False)

### собираем метрики 

In [ ]:
import datasets
test_dataset = datasets.Dataset.from_dict(test_set)
from ragas.metrics import (
    answer_relevancy,
    context_relevancy,
    answer_relevancy,
    answer_similarity
)
from ragas import evaluate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

result = evaluate(
    test_dataset,
    metrics=[
        context_relevancy,
        answer_relevancy,
        answer_similarity,
    ],
    llm=ChatOpenAI(model_name="gpt-3.5-turbo-0125"),
    embeddings=OpenAIEmbeddings()
)

In [ ]:
from src.metrics.metric_zoo import calculate_cosine_similarity_TF_IDF, calculate_similarity_spacy, calculate_bleu

In [ ]:
test_set['bleu_score'] = test_set.apply(calculate_bleu, axis=1)
test_set['sim-spacy'] = test_set.apply(calculate_similarity_spacy, axis=1)
test_set['cos-sim-TF-IDF'] = test_set.apply(calculate_cosine_similarity_TF_IDF, axis=1)

result['bleu_score'] = test_set['bleu_score'].mean()
result['sim-spacy'] = test_set['sim-spacy'].mean()
result['cos-sim-TF-IDF'] = test_set['cos-sim-TF-IDF'].mean()

In [ ]:
result  